In [44]:
-- setup some useful stuff
require 'nn'

-- help function to print in green
function cprint(str) print(sys.COLORS.green..str..'\27[0m') end

-- tests
precision = 1e-5
tester = torch.Tester()
function runTest(test)
    tester:add(test)
    tester:run()
    tester = torch.Tester()
end

##Linear Module

In [50]:
linearModule = function(inputSize, outputSize)
    
    -- module to build
    local this = {}
    
    -- standard deviation for initialization
    local stdv =  1./math.sqrt(outputSize)
    
    -- weight matrix
    this.weight = torch.Tensor(outputSize, inputSize):uniform(-stdv, stdv)
    
    -- bias vector
    this.bias = torch.Tensor(outputSize, 1):uniform(-stdv, stdv)
    
    -- forward operation
    this.forward = function(input)
        -- quirk to make it work with vectors and matrices
        input = (input:dim() == 1) and input:reshape(input:size(1), 1) or input
        -- multiply the input and weight matrix
        local output = this.weight * input
        -- add the expanded bias vector and return
        return output + this.bias:expand(output:size())
    end

    return this
end

runTest(function()
    print()
    local layer = linearModule(2,3)
    layer.weight:fill(2)
    layer.bias:fill(1)
    cprint('LinearModule weight is now:')
    print(layer.weight)
    cprint('LinearModule bias is now:')
    print(layer.bias)

    local inputVector = torch.Tensor(2):fill(-1)
    cprint('Testing input vector')
    print(inputVector)
    local output = layer.forward(inputVector)
    cprint('Output is:')
    print(output)
    local expected = torch.Tensor(3,1):fill(-3)
    tester:assertTensorEq(expected, output, precision)

    local inputMatrix = torch.Tensor(2, 3):fill(2)
    cprint('Testing input matrix')
    print(inputMatrix)
    local output = layer.forward(inputMatrix)
    cprint('Output is:')
    print(output)
    local expected = torch.Tensor(3,3):fill(9)
    tester:assertTensorEq(expected, output, precision)
end)

Running 1 tests	
|  ==> unknown
LinearModule weight is now:	
 2  2
 2  2
 2  2
[torch.DoubleTensor of dimension 3x2]

LinearModule bias is now:	
 1
 1
 1
[torch.DoubleTensor of dimension 3x1]

Testing input vector	
-1
-1
[torch.DoubleTensor of dimension 2]

Output is:	
-3
-3
-3
[torch.DoubleTensor of dimension 3x1]

Testing input matrix	
 2  2  2
 2  2  2
[torch.DoubleTensor of dimension 2x3]

Output is:	
 9  9  9
 9  9  9
 9  9  9
[torch.DoubleTensor of dimension 3x3]



_  ==> Done 

Completed 2 asserts in 1 tests with 0 errors	

--------------------------------------------------------------------------------	


# LogSoftMax
\begin{equation*}
   LogSoftMax(x_i) = -\ln \Bigl(\frac{1}{e^{x_i}} \sum_j e^{x_j}\Bigr)
\end{equation*}

In [47]:
logSoftMax = function(input) 
    
    -- quirk to make it work with vectors and matrices
    input = (input:dim() == 1) and input:reshape(input:size(1), 1) or input
    
    -- calculate sum of e^x_i and expand it to the right size
    local sumOfExp = torch.exp(input):sum(2):expand(input:size())
    
    -- calculate the rest of the formula and return
    return torch.exp(input):pow(-1):cmul(sumOfExp):log():mul(-1)
end


runTest(function()
    print()
    input = torch.range(1,9):reshape(3,3)
    cprint('Input matrix is:')
    print(input)
    local lms = nn.LogSoftMax()
    local expected = lms:forward(input)
    cprint('Output from nn.LogSoftMax is:')
    print(expected)
    local output = logSoftMax(input)
    cprint('Output from logSoftMax is:')
    print(output)
    tester:assertTensorEq(expected, output, 1e-5)
end)

Running 1 tests	
|  ==> unknown
Input matrix is:	
 1  2  3
 4  5  6
 7  8  9
[torch.DoubleTensor of dimension 3x3]



Output from nn.LogSoftMax is:	
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
[torch.DoubleTensor of dimension 3x3]

Output from logSoftMax is:	
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
-2.4076 -1.4076 -0.4076
[torch.DoubleTensor of dimension 3x3]

_  ==> Done 

Completed 1 asserts in 1 tests with 0 errors	

--------------------------------------------------------------------------------	


In [7]:


--- setting the random generator seed for reproducibility
torch.manualSeed(42)

-- network parameters
local embeddingSize = 4
local contextLength = 3
local vocabSize = 10
local hiddenSize = 5
local stdv = 1
-- Vocabulary

local lookupTable = torch.FloatTensor(embeddingSize, vocabSize):uniform(-stdv, stdv)
cprint('LookupTable of '..vocabSize..' words, each is a vector of size '..embeddingSize)
print(lookupTable)

-- Hidden

local contextToHidden = torch.FloatTensor(hiddenSize, contextLength * embeddingSize):uniform(-stdv, stdv)

cprint('Context-To-Hidden matrix is:')
print(contextToHidden)

local hiddenToEmbedding = torch.FloatTensor(embeddingSize, hiddenSize):uniform(-stdv, stdv)

cprint('Hidden-To-Embedding matrix is:')
print(hiddenToEmbedding)

local embeddingToVocabulary = torch.FloatTensor(vocabSize, embeddingSize):uniform(-stdv, stdv)

cprint('Embedding-To-Vocabulary matrix is:')
print(embeddingToVocabulary)

function softMax(matrix)
   -- -log(sum(exp(matrix)) * 1/exp(matrix))
   return torch.mul(torch.exp(matrix):pow(-1), torch.sum(torch.exp(matrix), 1)[1]):log():mul(-1)
end

-- Forward
--[[
local oneHot = torch.FloatTensor():eye(vocabSize)
cprint('1-Hot representation of second word is:')
print(oneHot[2])

local secondWord = torch.mv(lookupTable, oneHot[2])
cprint('Vector representation of second word is:')
print(secondWord)
--]]
local oneHotIndices = torch.LongTensor{1,3,5}
cprint('Context will be built of words at indices...')
print(oneHotIndices)

local contextMatrix = lookupTable:index(2, oneHotIndices)
cprint('...which corresponds to the following matrix:')
print(oneHotContext)
--[[
local contextMatrix = torch.mm(lookupTable, oneHotContext)
cprint('The corresponding matrix of embeddings are:')
print(contextMatrix)
--]]
local contextVector = torch.reshape(contextMatrix, contextMatrix:nElement())
cprint('...which reshaped as a vector is:')
print(contextVector)

local output = contextToHidden * contextVector
print(output)

output = torch.mv(hiddenToEmbedding, output)
print(output)

output = torch.mv(embeddingToVocabulary, output)
print(output)

output = softMax(output)
print(output)


LookupTable of 10 words, each is a vector of size 4	
-0.2509  0.5931  0.9014 -0.6331  0.4640  0.5594  0.1973  0.1937 -0.6880 -0.1083
-0.6880 -0.8001 -0.8838 -0.0815  0.7324 -0.3326  0.2022 -0.7143  0.4161  0.3018
-0.9588 -0.8872  0.9398  0.4440  0.6649  0.8771 -0.5753 -0.9984 -0.6364  0.9844
-0.6332  0.2350 -0.3915  0.2233  0.0495 -0.9859 -0.1361 -0.9539 -0.4175  0.0495
[torch.FloatTensor of dimension 4x10]

Context-To-Hidden matrix is:	
Columns 1 to 10
 0.2237 -0.2003 -0.7210 -0.9067 -0.4157  0.9475 -0.2673 -0.5345 -0.0879 -0.8188
-0.6007 -0.2351  0.0285  0.9665  0.1848 -0.0665 -0.9071  0.7199  0.2151  0.3606
-0.8699 -0.9735  0.8978  0.8844  0.9313  0.1266  0.6168 -0.2292 -0.3908 -0.9681
 0.3685 -0.5179 -0.1197  0.3665 -0.7559  0.2200 -0.0096  0.6664 -0.9312 -0.6533
-0.4824 -0.6355  0.3250  0.5107 -0.3766 -0.1497  0.0401 -0.5841  0.0934  0.1354

Columns 11 to 12
 0.5704  0.2368
-0.6590 -0.0990
-0.8047 -0.5382
 0.8186 -0.2179
-0.6303 -0.9374
[torch.FloatTensor of dimension 5x12]

Hidde